In [133]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import json

%matplotlib inline

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from keras.layers import *
from keras.optimizers import *
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons
from tensorflow.keras.optimizers import RMSprop,SGD





In [134]:
# Load .env environment variables
load_dotenv()

True

In [135]:
# Set Alpaca API key and secret
alpaca_api_key = "PKNHKA7JSFYR088WXJKA"
alpaca_secret_key = "YyABDvtTosEMl53YfKDoDTZJIK1QP4lpuXw4fFM0"

In [136]:
# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [137]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [138]:
# Set timeframe to '1D' pulling non-vol stock data
timeframe = "1D"

# Set start and end datetimes between now and 3 years ago.
start_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-06-01", tz="America/New_York").isoformat()
# Set the ticker info
tickers = ["SQ","IBM","ENPH"]


In [139]:
small_cap_tick = api.get_barset(
   tickers,
   timeframe,
   start=start_date,
   end=end_date,
   limit=1000
).df
# Display sample data
small_cap_tick.head()

ENPH                                    IBM  \
                            open   high    low   close   volume    open   
time                                                                      
2020-01-02 00:00:00-05:00  26.37  29.35  26.04  29.340  7582721  135.00   
2020-01-03 00:00:00-05:00  28.79  29.73  28.38  29.280  4682579  133.57   
2020-01-06 00:00:00-05:00  29.05  29.66  27.83  29.650  4462305  133.42   
2020-01-07 00:00:00-05:00  29.82  30.20  28.66  29.995  5082191  133.69   
2020-01-08 00:00:00-05:00  29.68  31.50  29.44  31.000  5240083  134.51   

                                                                 SQ         \
                             high       low   close   volume   open   high   
time                                                                         
2020-01-02 00:00:00-05:00  135.92  134.7701  135.46  2650485  62.99  64.05   
2020-01-03 00:00:00-05:00  134.86  133.5600  134.32  1870413  62.59  63.27   
2020-01-06 00:00:00-05:00  134.24  133.2000  134.08  2031658  61.36  62.58   
2020-01-07 00:00:00-05:00  134.96  133.5633  134.15  2631709  64.57  65.49   
2020-01-08 00:00:00-05:00  135.86  133.9200  135.30  3662956  64.56  68.00   

                                                   
                             low  close    volume  
time                                               
2020-01-02 00:00:00-05:00  62.95  63.83   4571798  
2020-01-03 00:00:00-05:00  62.33  63.00   4870465  
2020-01-06 00:00:00-05:00  61.13  62.57   5681852  
2020-01-07 00:00:00-05:00  63.66  64.57   8302437  
2020-01-08 00:00:00-05:00  64.15  67.60  11170180

In [140]:
groups = {"Small cap":["WKHS","BRP","DX"], "Mid cap":["RIOT","TTD","DXC"],"Large cap":["SQ","IBM","ENPH"]}

In [141]:
small_cap_tick.isna().sum()


ENPH  open      0
      high      0
      low       0
      close     0
      volume    0
IBM   open      0
      high      0
      low       0
      close     0
      volume    0
SQ    open      0
      high      0
      low       0
      close     0
      volume    0
dtype: int64

In [142]:
wtf = []

In [143]:
def window_data(Small_cap_tick, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(Small_cap_tick) - window - 1):
        features = Small_cap_tick.iloc[i:(i + window), feature_col_number]
        target = Small_cap_tick.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [144]:
#creating function to pull ticker names
def training_model(tickers):
    Small_cap_tick = api.get_barset(
       tickers,
       timeframe,
       start=start_date,
       end=end_date,
       limit=1000
    ).df
    # Display sample data
    Small_cap_tick.head()
    
    #
    data = np.array(Small_cap_tick)

    # train test split, we can take last 100 data points as test set
    train , test = data[0:-100], data[-100:]
    
   
        # Predict Closing Prices using a 10 day window of previous closing prices
    # Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
    window_size = 10

    # Column index 0 is the 'IBM ' column
    # Column index 1 is the `Close` column
    feature_column = 1
    target_column = 1
    X, y = window_data(Small_cap_tick, window_size, feature_column, target_column)
    wtf = X, y

        # Use 70% of the data for training and the remaineder for testing
    data_train = int(.7 * len(X))
    X_train = X[:data_train - 1]
    X_test = X[data_train:]
    y_train = y[:data_train - 1]
    y_test = y[data_train:]
    
        # Use the MinMaxScaler to scale data between 0 and 1.
    from sklearn.preprocessing import MinMaxScaler
    x_train_scaler = MinMaxScaler()
    x_test_scaler = MinMaxScaler()
    y_train_scaler = MinMaxScaler()
    y_test_scaler = MinMaxScaler()

    # Fit the scaler for the training Data
    x_train_scaler.fit(X_train)
    y_train_scaler.fit(y_train)

    # Scale the training data
    X_train = x_train_scaler.transform(X_train)
    y_train = y_train_scaler.transform(y_train)

    # Fit the scaler for the testing Data
    x_test_scaler.fit(X_test)
    y_test_scaler.fit(y_test)

    # Scale the y_test data
    X_test = x_test_scaler.transform(X_test)
    y_test = y_test_scaler.transform(y_test)

        # Reshape the features for the model
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Build the LSTM model. 
       
    # Note: Batching inputs has a different input shape of Samples/TimeSteps/Features
    model = Sequential()
    model.add(LSTM(
        units=30, return_sequences=True,
        input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=30, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=30))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    
    opt = SGD(lr=0.01, momentum=0.9, clipnorm=1.0)
    model.compile(loss='mean_squared_error', optimizer=opt)

    
        # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

        # Summarize the model
    #model.summary()
    
        # Train the model
    # Use at least 10 epochs
    model.fit(X_train, y_train, epochs=50, shuffle=False, batch_size=1,
        verbose=1)
    
        # Evaluate the model
    model.evaluate(X_test, y_test)
    print ("test loss")
    print (model.evaluate(X_test, y_test))
    
            # Make some predictions
    predicted = model.predict(X_test)

        # Recover the original prices instead of the scaled version
    predicted_prices = y_test_scaler.inverse_transform(predicted)
    real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))
    
        # Create a DataFrame of Real and Predicted values
    stocks = pd.DataFrame({
        "Real": real_prices.ravel(),
        "Predicted": predicted_prices.ravel()
    }, index = Small_cap_tick.index[-len(real_prices): ]) 
   
    
    return stocks


In [145]:
#training_model(Small_cap_tick)



In [146]:
#clipnorm=0.1


In [147]:


for(name,l) in groups.items():


    print (name)
    stocks = (training_model(l))
    plt.plot(stocks.index, stocks[["Real"]],label="Real")
    plt.plot(stocks.index, stocks[["Predicted"]], label="Predicted")
    plt.suptitle(name)
    plt.xlabel("datetime")
    plt.ylabel("Normalized closed price")
    plt.legend()
    plt.show()
    
    #print (training_model(l))
  

NameError: name 'model' is not defined